# **Customer Shopping Preference Dataset Transformation For Data Modelling (Kimball/Star Schema Approach)**

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('shopping_trends_updated_2.csv')
shop_df = df.copy()
shop_df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
shop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3900 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [113]:
shop_df.rename(columns={
    'Customer ID':'Customer_ID',
    'Item Purchased':'Item_Purchased',
    'Purchase Amount (USD)':'Purchase_Amount_USD',
    'Review Rating':'Review_Rating',
    'Subscription Status':'Subscription_Status',
    'Shipping Type':'Shipping_Type',
    'Discount Applied':'Discount_Applied',
    'Promo Code Used':'Promo_Code_Used',
    'Previous Purchases':'Previous_Purchases',
    'Payment Method':'Payment_Method',
    'Frequency of Purchases':'Frequency_of_Purchases'
    },inplace=True)

In [114]:
shop_df

,Customer_ID,Age,Gender,Item_Purchased,Category,Purchase_Amount_USD,Location,Size,Color,Season,Review_Rating,Subscription_Status,Shipping_Type,Discount_Applied,Promo_Code_Used,Previous_Purchases,Payment_Method,Frequency_of_Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly


In [115]:
shop_df.duplicated().value_counts()

False    3900
dtype: int64

**Customers:**

In [116]:
dim_customers = shop_df[['Customer_ID', 'Age', 'Gender']]
dim_customers

,Customer_ID,Age,Gender
0,1,55,Male
1,2,19,Male
2,3,50,Male
3,4,21,Male
4,5,45,Male
...,...,...,...
3895,3896,40,Female
3896,3897,52,Female
3897,3898,46,Female
3898,3899,44,Female


**Items:**

In [117]:
dim_items = shop_df.copy()[['Item_Purchased','Category','Size', 'Color']]
dim_items.drop_duplicates(inplace=True)
dim_items.reset_index(inplace=True)
dim_items.drop('index',axis=1,inplace=True)
dim_items['Item_ID'] = dim_items.index+1
dim_items = dim_items[['Item_ID','Item_Purchased','Category','Size', 'Color']]
dim_items

,Item_ID,Item_Purchased,Category,Size,Color
0,1,Blouse,Clothing,L,Gray
1,2,Sweater,Clothing,L,Maroon
2,3,Jeans,Clothing,S,Maroon
3,4,Sandals,Footwear,M,Maroon
4,5,Blouse,Clothing,M,Turquoise
...,...,...,...,...,...
1818,1819,Hat,Accessories,XL,Purple
1819,1820,Dress,Clothing,S,Yellow
1820,1821,Dress,Clothing,L,Peach
1821,1822,Hat,Accessories,L,White


**Locations:**

In [118]:
unique_loco = [n for n in shop_df['Location'].unique()]
loco_id = [x+1 for x in range(len(unique_loco))]
dim_locations = pd.DataFrame({
    'Location_ID':loco_id,
    'Location':unique_loco
})
dim_locations

,Location_ID,Location
0,1,Kentucky
1,2,Maine
2,3,Massachusetts
3,4,Rhode Island
4,5,Oregon
5,6,Wyoming
6,7,Montana
7,8,Louisiana
8,9,West Virginia
9,10,Missouri


**Payment Methods:**

In [119]:
unique_payment_methods = [n for n in shop_df['Payment_Method'].unique()]
payment_id = [x+1 for x in range(len(unique_payment_methods))]
dim_payment_methods = pd.DataFrame({
    'Payment_Method_ID':payment_id,
    "Payment_Method":unique_payment_methods
})
dim_payment_methods

,Payment_Method_ID,Payment_Method
0,1,Venmo
1,2,Cash
2,3,Credit Card
3,4,PayPal
4,5,Bank Transfer
5,6,Debit Card


**Shipping Types:**

In [120]:
unique_shipping_types = [n for n in shop_df['Shipping_Type'].unique()]
shipping_type_id = [x+1 for x in range(len(unique_shipping_types))]
dim_shipping_types = pd.DataFrame({
    'Shipping_Type_ID':shipping_type_id,
    'Shipping_Type':unique_shipping_types
})
dim_shipping_types

,Shipping_Type_ID,Shipping_Type
0,1,Express
1,2,Free Shipping
2,3,Next Day Air
3,4,Standard
4,5,2-Day Shipping
5,6,Store Pickup


**Fact Transaction Table:**

In [131]:
fct_transactions = shop_df.merge(dim_items,on=['Item_Purchased','Category','Size', 'Color'])\
.merge(dim_locations,on='Location')\
.merge(dim_payment_methods,on='Payment_Method')\
.merge(dim_shipping_types,on='Shipping_Type')\
[['Customer_ID','Item_ID', 'Location_ID',
  'Payment_Method_ID', 'Shipping_Type_ID',
  'Purchase_Amount_USD','Review_Rating',
  'Discount_Applied','Promo_Code_Used',
  'Subscription_Status','Previous_Purchases',
  'Frequency_of_Purchases','Season']]

fct_transactions['Transaction_ID'] = fct_transactions.index+1

fct_transactions = fct_transactions[
    ['Transaction_ID','Customer_ID','Item_ID', 
     'Location_ID','Payment_Method_ID', 'Shipping_Type_ID',
  'Purchase_Amount_USD','Review_Rating',
  'Discount_Applied','Promo_Code_Used',
  'Subscription_Status','Previous_Purchases',
  'Frequency_of_Purchases','Season']
  ]

fct_transactions

,Transaction_ID,Customer_ID,Item_ID,Location_ID,Payment_Method_ID,Shipping_Type_ID,Purchase_Amount_USD,Review_Rating,Discount_Applied,Promo_Code_Used,Subscription_Status,Previous_Purchases,Frequency_of_Purchases,Season
0,1,1,1,1,1,1,53,3.1,Yes,Yes,Yes,14,Fortnightly,Winter
1,2,2940,307,1,1,1,46,4.3,No,No,No,34,Weekly,Winter
2,3,3576,478,1,1,1,95,3.7,No,No,No,49,Bi-Weekly,Fall
3,4,2268,1395,1,1,1,49,3.8,No,No,No,37,Weekly,Spring
4,5,1157,885,12,1,1,21,4.8,Yes,Yes,No,6,Every 3 Months,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,2219,396,38,6,6,67,3.5,No,No,No,20,Quarterly,Winter
3896,3897,819,610,38,6,6,91,4.9,Yes,Yes,Yes,34,Monthly,Winter
3897,3898,3617,531,49,6,6,23,3.3,No,No,No,33,Fortnightly,Winter
3898,3899,1569,323,48,6,6,76,4.6,Yes,Yes,No,8,Monthly,Spring


**Write fact and dimension tables to CSV files:**

In [156]:
# df_interested = [fct_transactions,dim_customers,dim_items,dim_locations,dim_payment_methods,dim_shipping_types]
# path = 'C:\\Users\\CORE i3\\Desktop\\Data Engineering-Analytics Project\\Subsets CSV\\'
# file_names = ['fct_transactions','dim_customers','dim_items','dim_locations','dim_payment_methods','dim_shipping_types']

# for df,name in zip(df_interested,file_names):
#     df.to_csv(path+name+'.csv',index=False)